# NERVE 2

## Intro: install all the dependencies/packets/modules

In [1]:
!rm -r spaan
!rm -r NERVE
!git clone git://github.com/nicolagulmini/spaan
!git clone git://github.com/nicolagulmini/NERVE
!python -m pip install git+https://github.com/nicolagulmini/tmhmm.py
!pip install Bio
!pip install Pandas
!apt-get install ncbi-blast+ # for the autoimmunity module if we meant to use blastp to make the comparisons

rm: cannot remove 'spaan': No such file or directory
rm: cannot remove 'NERVE': No such file or directory
Cloning into 'spaan'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (195/195), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 195 (delta 109), reused 91 (delta 37), pack-reused 0
Receiving objects: 100% (195/195), 5.63 MiB | 28.14 MiB/s, done.
Resolving deltas: 100% (109/109), done.
Cloning into 'NERVE'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 117 (delta 52), reused 51 (delta 13), pack-reused 0
Receiving objects: 100% (117/117), 17.24 MiB | 18.27 MiB/s, done.
Resolving deltas: 100% (52/52), done.
  Cloning https://github.com/nicolagulmini/tmhmm.py to /tmp/pip-req-build-x_ipeuxl
  Running command git clone -q https://github.com/nicolagulmini/tmhmm.py /tmp/pip-req-build-x_ipeuxl
  Created wheel for tmhmm.py: filename=tmh

In [2]:
import tmhmm                                                # to predict transmembrane domains
from Bio import SeqIO                                       # to handle .fasta files
from Bio.Blast.Applications import NcbiblastpCommandline    # autoimmunity module, to query the local sapiens database
from Bio.ExPASy.ScanProsite import scan, read               # scan prosite functional domains information
from Bio.Blast import NCBIXML                               # to parse the peptides
from NERVE import Protein                                   # to contain proteins information
from tensorflow import keras                                # to use the spaan model (to predict the probability of a protein to be an adhesin)
import pandas                                               # to read mhcpep.csv and write the final report
from spaan.data_processing import *                     # to extract proteins features for the spaan model

## User parameters

In [3]:
p_ad_no_citoplasm_filter = 0.46
p_ad_extracellular_filter = 0.38
transmemb_dom_limit_filter = 2
e_value = 1e-10
similarity_function = 0.8
verbose = 0
GRAM = 'p' # for now we can mantain it 'p' as 'positive', then the user will have to put the right parameter! p = positive, n = negative, a = archea (for psortb)

## Define all the initial objects/paths

In [4]:
# main program

# TODO: check parameters, license ecc.

path_to_fastas = "./NERVE/data/tre.fasta"
path_to_another_proteome = "./NERVE/data/tmp.fasta"
list_of_fasta_proteins = list(SeqIO.parse(path_to_fastas, "fasta")) # put the right path

list_of_proteins = []
for p in list_of_fasta_proteins:
	p_id = p.id
	p_seq = p.seq
	list_of_proteins.append(Protein.Protein(p_id, p_seq))

if verbose > 0:
	for p in list_of_proteins:
		p.print_information()

# Subcelloc 

In [5]:
# psortb test
'''
6. Limitations
    6.1 Proteins resident at multiple localization sites: Many proteins can exist at multiple localization sites. Examples of such proteins include integral membrane proteins with large periplasmic domains, or autotransporters, which contain an outer membrane pore domain and a cleaved extracellular domain. The current version of PSORTb handles this situation by flagging proteins which show a distribution of localization scores favouring two sites, rather than one. It is important to examine the distribution of localization scores carefully in order to determine if your submitted protein may have multiple localization sites and if so, which two sites are involved.
    6.2 Lipoproteins: The current version of PSORTb does not detect lipoprotein motifs.
    6.3 Precision vs. Recall: PSORTb is designed to emphasize precision (or specificity) over recall (or sensitivity). Programs which make predictions at all costs often provide incorrect or incomplete results, which can be propagated through annotated databases, datasets and reports in the literature. We believe that a confident prediction is more valuable than any prediction, and we have designed the program to this end. Note, however, that a user may choose to use their own reduced cutoff score in generating final predictions.
'''
# use the psortb.py in python code that I found online (link in the comment of the file itself). Then try it with linux.

'\n6. Limitations\n    6.1 Proteins resident at multiple localization sites: Many proteins can exist at multiple localization sites. Examples of such proteins include integral membrane proteins with large periplasmic domains, or autotransporters, which contain an outer membrane pore domain and a cleaved extracellular domain. The current version of PSORTb handles this situation by flagging proteins which show a distribution of localization scores favouring two sites, rather than one. It is important to examine the distribution of localization scores carefully in order to determine if your submitted protein may have multiple localization sites and if so, which two sites are involved.\n    6.2 Lipoproteins: The current version of PSORTb does not detect lipoprotein motifs.\n    6.3 Precision vs. Recall: PSORTb is designed to emphasize precision (or specificity) over recall (or sensitivity). Programs which make predictions at all costs often provide incorrect or incomplete results, which ca

In [6]:
# Subcelloc module
# ... cerca di capire come chiamare psortb da qua
print("Warning: the PSORTB output has to be 'terse'!")

# final predictions parsing from the output of psortb program
filename = "20210903075318_psortb_gramneg.txt" # put the right filename from the previous (and still absent) lines of code
fp = open("./NERVE/data/"+filename, 'r')
lines = fp.readlines()
for i in range(1, len(lines)):
    attributes = lines[i].split(' ')
    if attributes[0] == list_of_proteins[i-1].id:
        last_attributes = attributes[len(attributes)-1].split('\t') # example: ['SV=2', 'Cytoplasmic', '9.97\n']
        list_of_proteins[i-1].localization = last_attributes[1] # if it gives errors, try last_attributes[len(last_attributes)-2]
fp.close()

# Adhesin

In [7]:
# Adhesin module

model = keras.models.load_model('./NERVE/espaan_model.h5') # insert the path to location!
for p in list_of_proteins:
	p.p_ad = float(model.predict([
			     np.array([aminoacids_frequencies(p.sequence)]),
			     np.array([multiplet_frequencies(p.sequence, 3)]),
			     np.array([multiplet_frequencies(p.sequence, 4)]),
			     np.array([multiplet_frequencies(p.sequence, 5)]),
			     np.array([dipeptide_frequencies(p.sequence)]),
			     np.array([charge_composition(p.sequence)]),
			     np.array([hydrophobic_composition(p.sequence)])
		     ]))

# Tmhelices

In [8]:
# Tmhelices module

for p in list_of_proteins:
    annotation, _ = tmhmm.predict(p.sequence)
    p.tmhmm_seq = annotation
    transmembrane_domains = 0
    for i in range(len(annotation)-1):
        if (annotation[i] == 'i' or annotation[i] == 'o') and annotation[i+1] == 'M':
            transmembrane_domains += 1
    p.transmembrane_doms = transmembrane_domains
for p in list_of_proteins:
    if p.transmembrane_doms > 2:
        p.transmembrane_flag = 1

/usr/local/lib/python3.7/dist-packages/tmhmm/api.py:20: RuntimeWarning: divide by zero encountered in log
  _, path = viterbi(sequence, *model)


# Autoimmunity

In [9]:
# Autoimmunity module

blastx_cline = NcbiblastpCommandline(query=path_to_fastas, db="./NERVE/sapiens_database/sapiens", evalue=e_value, outfmt=5, out="./sapiens.xml") # 5 is for xml 
stdout, stderr = blastx_cline()
# print the warning: "you can find a sapiens.xml file on your working directory which is the outputs of the autoimmunity module. Do not delete during the computation. After the computation it will be deleted in order to avoid future collisions."
# poi cancella l'output

In [10]:
# for each result in the .xml file...
for record in NCBIXML.parse(open("./sapiens.xml")):

    query_name = record.query.split(' ')[0] # take only the query id 
    #print("QUERY: " + record.query)

    # take the right candidate to update
    tmp_protein = list_of_proteins[0]
    for p in list_of_proteins:
        if p.id == query_name:
            tmp_protein = p

    #print("Check the id of the corresponding candidate: " + tmp_protein.id)
    # for each effective alignment between the tmp candidate and the human proteome
    for alignment in record.alignments:
        # collect all the interesting peptides
        for hsp in alignment.hsps:
            tmp_protein.list_of_shared_human_peps += Protein.Protein.hsp_match_parser(hsp.match)

    # print out the peptides (if there are any)
    '''
    if len(tmp_protein.list_of_shared_human_peps) == 0:
        print("No interesting peptides.")
    else:
        print("List of interesting peptides: " + str(tmp_protein.list_of_shared_human_peps))
    print()
    '''

In [11]:
# mhcpep pandas visualization

mhcpep = pandas.read_csv("./NERVE/mhcpep.csv")
# quali sono i peptidi?
# come opero il confronto?
# realizzare poi la somma della formula 

# Conservation (optional)

In [12]:
# (optional) Conservation module 
# mettere tutto in un if path_to_another_proteome is not None
#!declare -r path_to_another_proteome = "./NERVE/data/tmp.fasta"
# capire come definire una variable in shell
!makeblastdb -in "./NERVE/data/tre.fasta" -dbtype prot -parse_seqids -out "./compare_proteome/compare_proteome"



Building a new DB, current time: 09/16/2021 19:43:16
New DB name:   /content/compare_proteome/compare_proteome
New DB title:  ./NERVE/data/tre.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 3 sequences in 0.000518799 seconds.


In [14]:
blastx_cline = NcbiblastpCommandline(query=path_to_fastas, db="./compare_proteome/compare_proteome", evalue=e_value, outfmt=5, out="./comparison.xml") # 5 is for xml 
stdout, stderr = blastx_cline()

for record in NCBIXML.parse(open("./comparison.xml")):
    query_name = record.query.split(' ')[0] 

    tmp_protein = list_of_proteins[0]
    for p in list_of_proteins:
        if p.id == query_name:
            tmp_protein = p
    # max score between all the alignments
    max_score = 0
    for alignment in record.alignments:
        # collect all the interesting peptides
        for hsp in alignment.hsps:
            if hsp.score > max_score:
                max_score = hsp.score
    
    tmp_protein.conservation_score  = max_score

QUERY: sp|P06846|EBGR_ECOLI HTH-type transcriptional regulator EbgR OS=Escherichia coli (strain K12) OX=83333 GN=ebgR PE=4 SV=2
QUERY: sp|P0AAC4|YBHL_ECOLI Inner membrane protein YbhL OS=Escherichia coli (strain K12) OX=83333 GN=ybhL PE=1 SV=1
QUERY: sp|P0AE16|AMPG_ECOLI Anhydromuropeptide permease OS=Escherichia coli (strain K12) OX=83333 GN=ampG PE=1 SV=1


# Function

In [ ]:
# Function module
# (make blastp on uniprot... NOOOOOO) #!makeblastdb -in "./uniprot_sprot.fasta" -dbtype prot -parse_seqids -out "uniprot"

# prosite information
for p in list_of_proteins:
    handler = scan(p.sequence)
    p.scan_prosite_information = read(scan(p.sequence))


# try deepgo! https://deepgo.cbrc.kaust.edu.sa/deepgo/

# Select

In [ ]:
# Select module (filters)
# define the ranking

## Print out all the retrieved information about each candidate of the given proteome

In [15]:
for protein in list_of_proteins:
    protein.print_information()

Information about protein sp|P06846|EBGR_ECOLI:
   accession number = P06846
   length = 327
   localization = Cytoplasmic
   estimated probability to be an adhesin = 0.01151728630065918
   number of transmembrane domains = 0
   no interesting peptides shared with sapiens
   conservation with the given proteome blastp score = 1730.0
   putative function of the protein = not yet implemented

Information about protein sp|P0AAC4|YBHL_ECOLI:
   accession number = P0AAC4
   length = 234
   localization = CytoplasmicMembrane
   estimated probability to be an adhesin = 0.04193472862243652
   number of transmembrane domains = 7
   list of interesting peptides shared with sapiens = ['+L A++LYL', 'L A++LYLD', 'A++LYLD I', '++LYLD IN', '+LYLD INL', 'LYLD INLF', 'YLD INLFL', 'INLFL LLR']
   conservation with the given proteome blastp score = 1186.0
   putative function of the protein = not yet implemented

Information about protein sp|P0AE16|AMPG_ECOLI:
   accession number = P0AE16
   length = 491